# 미션 소개

기계 번역 실습으로, 한국어 문장을 영어로 번역하는 모델을 구축

총 3가지 모델(Seq2Seq 기본 모델, Attention 적용 모델)을 구현하고 학습시키며, 각 모델의 성능을 비교 분석

JSON 파일 형식으로 제공되며, 각 항목은 한국어 문장("ko")과 영어 번역문("mt")으로 구성

## 가이드라인
- 데이터 전처리
    - 적절한 토크나이저를 선택하여 한국어, 영어 문장을 토큰화하세요.
    - 문장 길이를 분석하여, 전체 문장의 길이에 맞게 최대 길이(MAX_LENGTH)를 설정하고, 필요한 경우 SOS, EOS, PAD, UNK 등의 특수 토큰을 정의한다.
- 어휘 사전 구축
    - 한국어와 영어 각각의 어후 사전을 구성
    - 단어의 등장 빈도를 고려하여, 추후 임베딩이나 기타 모델 구성에 활용할 수 있다.
- 텐서 변환 및 데이터 로더 구현
    - 각 문장을 토큰의 인덱스 시퀀스로 변환한 후, 고정 길이에 맞게 PAD 토큰으로 패딩한다.
    - `TensorDataset`과 `DataLoader`를 활용하여 학습 데이터를 배치 단위로 효율적으로 처리할 수 있도록 구현
- 모델 구현 및 실습
    - Seq2Seq 모델
        - 기본 GRU 기반의 Encoder-Decoder 모델을 구현하고, Teacher Forcing 기법을 적용해 학습한다.
    - Attention 모델
        - Bahdanau Attention(Bahdanau 혹은 Luong)을 적용한 디코더를 구현하여, 번역 성능을 높이기
- 모델 학습 및 평가
    - 각 모델별로 학습을 진행한 후, 평가 함수를 활용하여 번역 결과를 확인한다.
    - 무작위 문장 쌍에 대해 모델의 변역 결과를 출력하고, 출력 문장을 정제(특수 토큰 제거 등)하여 성능을 분석
- 모델 성능 개선 (심화)
    - 데이터 전처리 방법 개선(예:불용어 제거, 정규화 등)
    - 모델 구조 변경(레이어 수, 은닉 상태 크기, Attention 기법 수정 등)
    - 하이퍼파라미터 튜닝(학습률, 배치 크기 등)
    - 다양한 평가 지표(예: BLEU 점수) 도입

# 환경 설정

In [1]:
!apt-get install -y mecab mecab-ipadic-utf8 libmecab-dev
!pip install konlpy

zsh:1: command not found: apt-get


In [2]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
import random
import pickle
from tqdm import tqdm
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from konlpy.tag import Okt
import nltk
from nltk.tokenize import word_tokenize

## GPU 세팅

In [3]:
print("PyTorch:", torch.__version__)
print("MPS available:", torch.backends.mps.is_available())
print("CUDA available:", torch.cuda.is_available())

if torch.backends.mps.is_available():
    device = torch.device("mps")  # 맥북 M1/M2 GPU
    print("Using MPS (Apple Silicon GPU)")
elif torch.cuda.is_available():
    device = torch.device("cuda")  # NVIDIA GPU (Colab, Windows 등)
    print("Using CUDA (NVIDIA GPU)")
else:
    device = torch.device("cpu")   # CPU fallback
    print("Using CPU")

print("Selected device:", device)

PyTorch: 2.8.0
MPS available: True
CUDA available: False
Using MPS (Apple Silicon GPU)
Selected device: mps


## 한국어 세팅

In [4]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action='ignore')

# 나눔 폰트가 없는 경우를 대비한 대체 폰트 설정
try:
    # 나눔 폰트가 설치되어 있는지 확인
    path = '../NanumGothic.ttf'
    font_name = fm.FontProperties(fname=path, size=10).get_name()
    plt.rc('font', family=font_name)
    fm.fontManager.addfont(path)
    print("나눔 폰트를 성공적으로 설정했습니다.")
except:
    # 나눔 폰트가 없는 경우 기본 폰트 사용
    print("나눔 폰트를 찾을 수 없습니다. 기본 폰트를 사용합니다.")
    plt.rc('font', family='DejaVu Sans')  # 기본 폰트

나눔 폰트를 성공적으로 설정했습니다.


# 데이터

## NLTK

In [5]:
# NLTK 데이터를 현재 폴더의 nltk_data 디렉토리에 다운로드
current_dir = os.getcwd()
nltk_data_dir = os.path.join(current_dir, 'nltk_data')
nltk.data.path.append(nltk_data_dir)

print(f"NLTK 데이터 경로: {nltk_data_dir}")
print(f"경로 존재 여부: {os.path.exists(nltk_data_dir)}")

nltk.download('punkt', download_dir=nltk_data_dir)
nltk.download('punkt_tab', download_dir=nltk_data_dir)

NLTK 데이터 경로: /Users/leeyoungho/develop/ai_study/mission/mission11/nltk_data
경로 존재 여부: True


[nltk_data] Downloading package punkt to /Users/leeyoungho/develop/ai_
[nltk_data]     study/mission/mission11/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/leeyoungho/develop
[nltk_data]     /ai_study/mission/mission11/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## 시드 설정

In [6]:
RANDOM_STATE = 42
random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)

## 데이터셋 불러오기

In [7]:
train_json_file_path = "./일상생활및구어체_한영/일상생활및구어체_한영_train_set.json"
valid_json_file_path = "./일상생활및구어체_한영/일상생활및구어체_한영_valid_set.json"

# 결과 저장 폴더 생성
os.makedirs("checkpoints", exist_ok=True)
os.makedirs("results", exist_ok=True)
os.makedirs("models", exist_ok=True)

print("데이터 경로:")
print(f"훈련 데이터: {train_json_file_path}")
print(f"검증 데이터: {valid_json_file_path}")

데이터 경로:
훈련 데이터: ./일상생활및구어체_한영/일상생활및구어체_한영_train_set.json
검증 데이터: ./일상생활및구어체_한영/일상생활및구어체_한영_valid_set.json


In [8]:
def load_json(file_path, max_samples=None):
    """JSON 파일을 로드하고 데이터 추출"""
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        if max_samples:
            data = data["data"][:max_samples]
        else:
            data = data["data"]
            
        print(f"로드된 데이터 개수: {len(data)}")
        return data
    except Exception as e:
        print(f"데이터 로딩 오류: {e}")
        return []

# 데이터 로드
print("=== 데이터 로딩 중 ===")
data_train = load_json(train_json_file_path, max_samples=10000)
data_valid = load_json(valid_json_file_path, max_samples=1000)

# ko와 mt 데이터 추출
ko_sentences_train = [item["ko"] for item in data_train]
mt_sentences_train = [item["mt"] for item in data_train]
ko_sentences_valid = [item["ko"] for item in data_valid]
mt_sentences_valid = [item["mt"] for item in data_valid]

print(f"훈련 데이터: 한국어 {len(ko_sentences_train)}개, 영어 {len(mt_sentences_train)}개")
print(f"검증 데이터: 한국어 {len(ko_sentences_valid)}개, 영어 {len(mt_sentences_valid)}개")

=== 데이터 로딩 중 ===
로드된 데이터 개수: 10000
로드된 데이터 개수: 1000
훈련 데이터: 한국어 10000개, 영어 10000개
검증 데이터: 한국어 1000개, 영어 1000개


In [9]:
if len(data_train) > 0:
    print(f"Key: {list(data_train[0].keys())}")
    print(f"Value: {data_train[0]}")
    print()

if len(data_valid) > 0:
    print(f"Key: {list(data_valid[0].keys())}")
    print(f"Value: {data_valid[0]}")

Key: ['sn', 'data_set', 'domain', 'subdomain', 'ko_original', 'ko', 'mt', 'en', 'source_language', 'target_language', 'word_count_ko', 'word_count_en', 'word_ratio', 'file_name', 'source', 'license', 'style', 'included_unknown_words', 'ner']
Value: {'sn': 'INTSALDSUT062119042703238', 'data_set': '일상생활및구어체', 'domain': '해외영업', 'subdomain': '도소매유통', 'ko_original': '원하시는 색상을 회신해 주시면 바로 제작 들어가겠습니다.', 'ko': '원하시는 색상을 회신해 주시면 바로 제작 들어가겠습니다.', 'mt': 'If you reply to the color you want, we will start making it right away.', 'en': 'If you reply to the color you want, we will start making it right away.', 'source_language': 'ko', 'target_language': 'en', 'word_count_ko': 7, 'word_count_en': 15, 'word_ratio': 2.143, 'file_name': 'INTSAL_DSUT.xlsx', 'source': '크라우드소싱', 'license': 'open', 'style': '구어체', 'included_unknown_words': False, 'ner': None}

Key: ['sn', 'data_set', 'domain', 'subdomain', 'ko_original', 'ko', 'mt', 'en', 'source_language', 'target_language', 'word_count_ko', 'word_count_en',

In [10]:
# 데이터 샘플 확인
print("=== 데이터 샘플 ===")
for i in range(min(5, len(data_train))):
    print(f"샘플 {i+1}:")
    print(f"  한국어: {ko_sentences_train[i]}")
    print(f"  영어: {mt_sentences_train[i]}")
    print()

=== 데이터 샘플 ===
샘플 1:
  한국어: 원하시는 색상을 회신해 주시면 바로 제작 들어가겠습니다.
  영어: If you reply to the color you want, we will start making it right away.

샘플 2:
  한국어: 형님 제일 웃긴 그림이 뭔지 알아요.
  영어: I know what the funniest picture is.

샘플 3:
  한국어: >속옷을?
  영어: Underwear?

샘플 4:
  한국어: 그래도 가격이 꽤 비싸니까 많이 살게요.
  영어: However, the price is quite high, so I will buy a lot.

샘플 5:
  한국어: AAA님, 제가 회의에서 화를 냈던 점 정말 사과드리고 싶습니다.
  영어: AAA, I really want to apologize for being upset at the meeting.



In [11]:
# 한국어 및 영어 토크나이저 설정
tokenizer_ko = Okt().morphs
tokenizer_en = word_tokenize

# 토크나이저 테스트
print("=== 토크나이저 테스트 ===")
test_ko = "안녕하세요 오늘 날씨가 좋네요"
test_en = "Hello, how are you today?"

print(f"한국어 토큰화: {test_ko} -> {tokenizer_ko(test_ko)}")
print(f"영어 토큰화: {test_en} -> {tokenizer_en(test_en)}")

=== 토크나이저 테스트 ===
한국어 토큰화: 안녕하세요 오늘 날씨가 좋네요 -> ['안녕하세요', '오늘', '날씨', '가', '좋네요']
영어 토큰화: Hello, how are you today? -> ['Hello', ',', 'how', 'are', 'you', 'today', '?']


In [12]:
# 문장 길이 분석
print("=== 문장 길이 분석 ===")
ko_lengths = [len(tokenizer_ko(sent)) for sent in ko_sentences_train]
en_lengths = [len(tokenizer_en(sent)) for sent in mt_sentences_train]

print(f"한국어 문장 길이 통계:")
print(f"  최소: {min(ko_lengths)}")
print(f"  최대: {max(ko_lengths)}")
print(f"  평균: {np.mean(ko_lengths):.2f}")
print(f"  중간값: {np.median(ko_lengths):.2f}")

print(f"\n영어 문장 길이 통계:")
print(f"  최소: {min(en_lengths)}")
print(f"  최대: {max(en_lengths)}")
print(f"  평균: {np.mean(en_lengths):.2f}")
print(f"  중간값: {np.median(en_lengths):.2f}")

# MAX_LENGTH 설정 (95% 백분위수 기준)
ko_95th = np.percentile(ko_lengths, 95)
en_95th = np.percentile(en_lengths, 95)
MAX_LENGTH = int(max(ko_95th, en_95th)) + 2  # SOS, EOS 토큰 포함

print(f"\n설정된 MAX_LENGTH: {MAX_LENGTH}")

=== 문장 길이 분석 ===
한국어 문장 길이 통계:
  최소: 1
  최대: 51
  평균: 11.27
  중간값: 10.00

영어 문장 길이 통계:
  최소: 1
  최대: 51
  평균: 11.64
  중간값: 11.00

설정된 MAX_LENGTH: 25


In [13]:
# 특수 토큰 정의
SOS_token = 0  # Start of Sentence
EOS_token = 1  # End of Sentence
PAD_token = 2  # Padding
UNK_token = 3  # Unknown

print("=== 특수 토큰 정의 ===")
print(f"SOS_token: {SOS_token}")
print(f"EOS_token: {EOS_token}")
print(f"PAD_token: {PAD_token}")
print(f"UNK_token: {UNK_token}")

=== 특수 토큰 정의 ===
SOS_token: 0
EOS_token: 1
PAD_token: 2
UNK_token: 3


# 어휘 사전

## Lang 클래스 구현

In [14]:
class Lang:
    def __init__(self, name):
        self.name = name
        # 초기에는 PAD, SOS, EOS, UNK 토큰을 미리 등록
        self.word2index = {"PAD": PAD_token, "SOS": SOS_token, "EOS": EOS_token, "<unk>": UNK_token}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS", UNK_token: "<unk>"}
        # word2count도 특수 토큰으로 초기화
        self.word2count = {"PAD": 0, "SOS": 0, "EOS": 0, "<unk>": 0}
        self.n_words = 4  # PAD, SOS, EOS, UNK 포함

    def addSentence(self, sentence, tokenizer):
        """문장을 토큰화하여 어휘 사전에 추가"""
        for word in tokenizer(sentence):
            self.addWord(word)

    def addWord(self, word):
        """단어를 어휘 사전에 추가"""
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.word2count[word] = 1
            self.n_words += 1
        else:
            self.word2count[word] += 1

    def get_vocab_size(self):
        """어휘 크기 반환"""
        return self.n_words

    def get_word_frequency(self, min_freq=1):
        """최소 빈도수 이상의 단어만 반환"""
        return {word: count for word, count in self.word2count.items() if count >= min_freq}

# Lang 클래스 테스트
print("=== Lang 클래스 테스트 ===")
test_lang = Lang("test")
test_sentence = "안녕하세요 반갑습니다"
test_lang.addSentence(test_sentence, tokenizer_ko)

print(f"어휘 크기: {test_lang.get_vocab_size()}")
print(f"단어-인덱스: {test_lang.word2index}")
print(f"인덱스-단어: {test_lang.index2word}")
print(f"단어 빈도수: {test_lang.word2count}")

=== Lang 클래스 테스트 ===
어휘 크기: 6
단어-인덱스: {'PAD': 2, 'SOS': 0, 'EOS': 1, '<unk>': 3, '안녕하세요': 4, '반갑습니다': 5}
인덱스-단어: {2: 'PAD', 0: 'SOS', 1: 'EOS', 3: '<unk>', 4: '안녕하세요', 5: '반갑습니다'}
단어 빈도수: {'PAD': 0, 'SOS': 0, 'EOS': 0, '<unk>': 0, '안녕하세요': 1, '반갑습니다': 1}


## 어휘 사전 구축

In [15]:
def prepareData(lang1, lang2, tokenizer1, tokenizer2, min_freq=2):
    """한국어와 영어 어휘 사전 구축"""
    print("=== 어휘 사전 구축 중 ===")
    
    input_lang = Lang(lang1)  # 한국어
    output_lang = Lang(lang2)  # 영어
    
    print("한국어 문장 처리 중...")
    for sentence in tqdm(ko_sentences_train, desc="한국어 처리"):
        input_lang.addSentence(sentence, tokenizer1)
    
    print("영어 문장 처리 중...")
    for sentence in tqdm(mt_sentences_train, desc="영어 처리"):
        output_lang.addSentence(sentence, tokenizer2)
    
    # 최소 빈도수 필터링
    print(f"\n=== 어휘 통계 ===")
    print(f"한국어 어휘 크기: {input_lang.get_vocab_size()}")
    print(f"영어 어휘 크기: {output_lang.get_vocab_size()}")
    
    # 빈도수 상위 10개 단어 확인 (특수 토큰 제외)
    ko_top_words = [(word, count) for word, count in input_lang.word2count.items() 
                    if word not in ["PAD", "SOS", "EOS", "<unk>"]]
    ko_top_words = sorted(ko_top_words, key=lambda x: x[1], reverse=True)[:10]
    
    en_top_words = [(word, count) for word, count in output_lang.word2count.items() 
                    if word not in ["PAD", "SOS", "EOS", "<unk>"]]
    en_top_words = sorted(en_top_words, key=lambda x: x[1], reverse=True)[:10]
    
    print(f"\n한국어 상위 10개 단어: {ko_top_words}")
    print(f"영어 상위 10개 단어: {en_top_words}")
    
    return input_lang, output_lang

# 어휘 사전 구축
input_lang, output_lang = prepareData("한국어", "영어", tokenizer_ko, tokenizer_en, min_freq=2)

=== 어휘 사전 구축 중 ===
한국어 문장 처리 중...


한국어 처리: 100%|██████████| 10000/10000 [00:07<00:00, 1343.31it/s]


영어 문장 처리 중...


영어 처리: 100%|██████████| 10000/10000 [00:00<00:00, 33384.52it/s]



=== 어휘 통계 ===
한국어 어휘 크기: 13774
영어 어휘 크기: 9941

한국어 상위 10개 단어: [('.', 8520), ('을', 2868), ('이', 2532), ('>', 2398), (',', 2325), ('에', 2082), ('를', 1585), ('의', 1519), ('가', 1481), ('는', 1269)]
영어 상위 10개 단어: [('.', 8643), (',', 4518), ('the', 3815), ('to', 2839), ('I', 2490), ('you', 2371), ('a', 2272), ('and', 1850), ('>', 1823), ('it', 1696)]


## 어휘 사전 저장

In [16]:
# 어휘 사전 저장 (나중에 재사용하기 위해)
def save_vocab(lang, filename):
    """어휘 사전을 파일로 저장"""
    vocab_data = {
        'word2index': lang.word2index,
        'index2word': lang.index2word,
        'word2count': lang.word2count,
        'n_words': lang.n_words
    }
    
    with open(filename, 'wb') as f:
        pickle.dump(vocab_data, f)
    print(f"어휘 사전 저장 완료: {filename}")

def load_vocab(filename):
    """저장된 어휘 사전을 파일에서 로드"""
    with open(filename, 'rb') as f:
        vocab_data = pickle.load(f)
    
    lang = Lang("loaded")
    lang.word2index = vocab_data['word2index']
    lang.index2word = vocab_data['index2word']
    lang.word2count = vocab_data['word2count']
    lang.n_words = vocab_data['n_words']
    
    print(f"어휘 사전 로드 완료: {filename}")
    return lang

# 어휘 사전 저장
save_vocab(input_lang, "models/korean_vocab.pkl")
save_vocab(output_lang, "models/english_vocab.pkl")

어휘 사전 저장 완료: models/korean_vocab.pkl
어휘 사전 저장 완료: models/english_vocab.pkl


# 데이터 전처리 및 텐서 변환

## s2i, i2s

In [17]:
def sentence_to_indexes(sentence, lang, tokenizer, max_length):
    """문장을 토큰 인덱스로 변환"""
    tokens = tokenizer(sentence)
    indexes = [lang.word2index.get(token, UNK_token) for token in tokens]
    
    # SOS, EOS 토큰 추가
    indexes = [SOS_token] + indexes + [EOS_token]
    
    # 패딩
    if len(indexes) < max_length:
        indexes += [PAD_token] * (max_length - len(indexes))
    else:
        indexes = indexes[:max_length-1] + [EOS_token]
    
    return indexes

def indexes_to_sentence(indexes, lang):
    """인덱스를 문장으로 변환"""
    words = []
    for idx in indexes:
        if idx == PAD_token:
            continue
        if idx == EOS_token:
            break
        if idx in [SOS_token, PAD_token]:
            continue
        words.append(lang.index2word.get(idx, "<unk>"))
    return " ".join(words)

# 변환 함수 테스트
print("=== 변환 함수 테스트 ===")
test_ko = "안녕하세요 반갑습니다"
test_indexes = sentence_to_indexes(test_ko, input_lang, tokenizer_ko, MAX_LENGTH)
test_reconstructed = indexes_to_sentence(test_indexes, input_lang)

print(f"원본: {test_ko}")
print(f"토큰화: {tokenizer_ko(test_ko)}")
print(f"인덱스: {test_indexes}")
print(f"재구성: {test_reconstructed}")

=== 변환 함수 테스트 ===
원본: 안녕하세요 반갑습니다
토큰화: ['안녕하세요', '반갑습니다']
인덱스: [0, 211, 5187, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
재구성: 안녕하세요 반갑습니다


## 데이터셋 및 Dataloader 구현

In [18]:
class TranslationDataset:
    def __init__(self, ko_sentences, en_sentences, input_lang, output_lang, 
                 tokenizer_ko, tokenizer_en, max_length):
        self.ko_sentences = ko_sentences
        self.en_sentences = en_sentences
        self.input_lang = input_lang
        self.output_lang = output_lang
        self.tokenizer_ko = tokenizer_ko
        self.tokenizer_en = tokenizer_en
        self.max_length = max_length
        
    def __len__(self):
        return len(self.ko_sentences)
    
    def __getitem__(self, idx):
        ko_sentence = self.ko_sentences[idx]
        en_sentence = self.en_sentences[idx]
        
        # 한국어와 영어 문장을 인덱스로 변환
        ko_indexes = sentence_to_indexes(ko_sentence, self.input_lang, 
                                       self.tokenizer_ko, self.max_length)
        en_indexes = sentence_to_indexes(en_sentence, self.output_lang, 
                                       self.tokenizer_en, self.max_length)
        
        return {
            'ko': torch.tensor(ko_indexes, dtype=torch.long),
            'en': torch.tensor(en_indexes, dtype=torch.long),
            'ko_length': len(self.tokenizer_ko(ko_sentence)),
            'en_length': len(self.tokenizer_en(en_sentence))
        }

# 데이터셋 생성
print("=== 데이터셋 생성 중 ===")
train_dataset = TranslationDataset(
    ko_sentences_train, mt_sentences_train, 
    input_lang, output_lang, 
    tokenizer_ko, tokenizer_en, MAX_LENGTH
)

valid_dataset = TranslationDataset(
    ko_sentences_valid, mt_sentences_valid, 
    input_lang, output_lang, 
    tokenizer_ko, tokenizer_en, MAX_LENGTH
)

print(f"훈련 데이터셋 크기: {len(train_dataset)}")
print(f"검증 데이터셋 크기: {len(valid_dataset)}")

# 데이터셋 테스트
print("\n=== 데이터셋 테스트 ===")
sample = train_dataset[0]
print(f"샘플 데이터:")
print(f"  한국어 텐서: {sample['ko']}")
print(f"  영어 텐서: {sample['en']}")
print(f"  한국어 길이: {sample['ko_length']}")
print(f"  영어 길이: {sample['en_length']}")

=== 데이터셋 생성 중 ===
훈련 데이터셋 크기: 10000
검증 데이터셋 크기: 1000

=== 데이터셋 테스트 ===
샘플 데이터:
  한국어 텐서: tensor([ 0,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,  1,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2])
  영어 텐서: tensor([ 0,  4,  5,  6,  7,  8,  9,  5, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
         1,  2,  2,  2,  2,  2,  2])
  한국어 길이: 11
  영어 길이: 17


## DataLoader 생성

In [19]:
# DataLoader 생성
BATCH_SIZE = 32

def collate_fn(batch):
    """배치 데이터를 정리하는 함수"""
    ko = torch.stack([b['ko'] for b in batch])      # [B, S]
    en = torch.stack([b['en'] for b in batch])      # [B, T]
    ko_raw = torch.tensor([b['ko_length'] for b in batch], dtype=torch.long)
    en_raw = torch.tensor([b['en_length'] for b in batch], dtype=torch.long)

    # BOS/EOS(+2) 반영 후 실제 시퀀스 길이로 clamp
    ko_lengths = torch.clamp(ko_raw + 2, max=ko.size(1))
    en_lengths = torch.clamp(en_raw + 2, max=en.size(1))

    return {'ko': ko, 'en': en, 'ko_lengths': ko_lengths, 'en_lengths': en_lengths}


# 훈련 및 검증 DataLoader 생성
train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    collate_fn=collate_fn,
    num_workers=0  # Windows에서는 0으로 설정
)

valid_loader = DataLoader(
    valid_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    collate_fn=collate_fn,
    num_workers=0
)

print("=== DataLoader 생성 완료 ===")
print(f"훈련 배치 수: {len(train_loader)}")
print(f"검증 배치 수: {len(valid_loader)}")

# DataLoader 테스트
print("\n=== DataLoader 테스트 ===")
for batch in train_loader:
    print(f"배치 크기:")
    print(f"  한국어: {batch['ko'].shape}")
    print(f"  영어: {batch['en'].shape}")
    print(f"  한국어 길이: {batch['ko_lengths'][:5]}...")  # 처음 5개만
    print(f"  영어 길이: {batch['en_lengths'][:5]}...")    # 처음 5개만
    break

=== DataLoader 생성 완료 ===
훈련 배치 수: 313
검증 배치 수: 32

=== DataLoader 테스트 ===
배치 크기:
  한국어: torch.Size([32, 25])
  영어: torch.Size([32, 25])
  한국어 길이: tensor([ 8, 20,  5, 13,  9])...
  영어 길이: tensor([10, 19,  5, 10, 12])...


# Seq2Seq 모델 구현

## Encoder 클래스 구현

In [20]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, embedding_size, num_layers=1, dropout=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # 임베딩 레이어
        self.embedding = nn.Embedding(input_size, embedding_size, padding_idx=PAD_token)
        
        # GRU 레이어
        self.gru = nn.GRU(
            embedding_size, 
            hidden_size, 
            num_layers=num_layers, 
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # 드롭아웃
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input_seq, input_lengths, hidden=None):
        # 임베딩 적용
        embedded = self.dropout(self.embedding(input_seq))
        
        # PackedSequence로 변환 (패딩 제거)
        packed = nn.utils.rnn.pack_padded_sequence(
            embedded, input_lengths, batch_first=True, enforce_sorted=False
        )
        
        # GRU 순전파
        outputs, hidden = self.gru(packed, hidden)
        
        # PackedSequence를 다시 일반 텐서로 변환
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)
        
        return outputs, hidden
    
    def initHidden(self, batch_size):
        """은닉 상태 초기화"""
        return torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device)

# Encoder 테스트
print("=== Encoder 테스트 ===")
encoder = EncoderRNN(
    input_size=input_lang.get_vocab_size(),
    hidden_size=256,
    embedding_size=128,
    num_layers=2,
    dropout=0.1
).to(device)

# 테스트 입력
test_batch_size = 2
test_seq_len = 10
test_input = torch.randint(0, 100, (test_batch_size, test_seq_len)).to(device)
test_lengths = [8, 6]  # 첫 번째는 길이 8, 두 번째는 길이 6

# Encoder 순전파
encoder.eval()
with torch.no_grad():
    outputs, hidden = encoder(test_input, test_lengths)
    
print(f"입력 크기: {test_input.shape}")
print(f"출력 크기: {outputs.shape}")
print(f"은닉 상태 크기: {hidden.shape}")

=== Encoder 테스트 ===
입력 크기: torch.Size([2, 10])
출력 크기: torch.Size([2, 8, 256])
은닉 상태 크기: torch.Size([2, 2, 256])


## Attention Decoder 클래스 구현

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [22]:
class AttentionDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, embedding_size, num_layers=1, dropout=0.1, pad_idx=None, device="cpu"):
        super(AttentionDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.device = device
        self.pad_idx = pad_idx

        # 임베딩
        self.embedding = nn.Embedding(output_size, embedding_size, padding_idx=pad_idx)

        # Attention 레이어들
        self.attention = nn.Linear(hidden_size * 2, 1)
        self.attention_combine = nn.Linear(embedding_size + hidden_size, hidden_size)

        # GRU
        self.gru = nn.GRU(
            input_size=hidden_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
        )

        # 출력층
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_seq, hidden, encoder_outputs):
        B = input_seq.size(0)
        
        # 1) 임베딩
        embedded = self.dropout(self.embedding(input_seq))  # (B,1,E)
        
        # 2) Attention 가중치 계산 - 수정된 버전
        attention_scores = self.calculate_attention(hidden, encoder_outputs)  # (B,S)
        attention_weights = F.softmax(attention_scores, dim=1)  # (B,S)
        
        # 3) Context vector 계산
        context = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs)  # (B,1,H)
        
        # 4) 임베딩과 컨텍스트 결합
        gru_input = torch.cat((embedded, context), dim=2)  # (B,1,E+H)
        gru_input = self.attention_combine(gru_input)      # (B,1,H)
        
        # 5) GRU 진행
        output, hidden = self.gru(gru_input, hidden)
        
        # 6) 어휘 분포
        output = self.out(output)
        
        return output, hidden, attention_weights.unsqueeze(1)

    def calculate_attention(self, hidden, encoder_outputs):
        """수정된 Attention 계산"""
        B, S, H = encoder_outputs.size()
        
        # hidden의 마지막 레이어만 사용
        if hidden.dim() == 3:  # (num_layers, B, H)
            dec_h_t = hidden[-1]  # (B,H)
        else:  # (B, H)
            dec_h_t = hidden
            
        # Attention 스코어 계산
        dec_h_expanded = dec_h_t.unsqueeze(1).expand(-1, S, -1)  # (B,S,H)
        concat_input = torch.cat([dec_h_expanded, encoder_outputs], dim=2)  # (B,S,2H)
        scores = self.attention(concat_input).squeeze(-1)  # (B,S)
        
        return scores

    def initHidden(self, batch_size):
        return torch.zeros(self.num_layers, batch_size, self.hidden_size, device=self.device)

In [23]:
print("=== Attention Decoder 재생성 (최종 수정된 버전) ===")
decoder = AttentionDecoderRNN(
    hidden_size=256,
    output_size=output_lang.get_vocab_size(),
    embedding_size=128,
    num_layers=2,
    dropout=0.1,
    pad_idx=PAD_token,
    device=device,
).to(device)

=== Attention Decoder 재생성 (최종 수정된 버전) ===


## Seq2Seq 모델 클래스 구현

In [24]:
class Seq2SeqModel(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2SeqModel, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, ko_input, ko_lengths, en_input, teacher_forcing_ratio=0.5):
        batch_size = ko_input.size(0)
        max_length = en_input.size(1)
        vocab_size = self.decoder.output_size
        
        # 인코더 순전파
        encoder_outputs, encoder_hidden = self.encoder(ko_input, ko_lengths)
        
        # 디코더 초기화
        decoder_input = torch.full((batch_size, 1), SOS_token, dtype=torch.long, device=self.device)
        decoder_hidden = encoder_hidden
        
        # 출력 저장용
        outputs = torch.zeros(batch_size, max_length, vocab_size, device=self.device)
        
        # Teacher Forcing 적용
        use_teacher_forcing = random.random() < teacher_forcing_ratio
        
        for t in range(max_length):
            # 디코더 순전파
            output, decoder_hidden, attention = self.decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            
            # 출력 저장
            outputs[:, t, :] = output[:, 0, :]
            
            # Teacher Forcing 비율을 점진적으로 감소
            if use_teacher_forcing and t < max_length - 1:
                decoder_input = en_input[:, t:t+1]
            else:
                top1 = output[:, 0, :].argmax(1)
                decoder_input = top1.unsqueeze(1)
        
        return outputs
    
    def predict(self, ko_input, ko_lengths, max_length=50):
        """추론 시 사용하는 함수"""
        batch_size = ko_input.size(0)
        vocab_size = self.decoder.output_size
        
        # 인코더 순전파
        encoder_outputs, encoder_hidden = self.encoder(ko_input, ko_lengths)
        
        # 디코더 초기화
        decoder_input = torch.full((batch_size, 1), SOS_token, dtype=torch.long, device=self.device)
        decoder_hidden = encoder_hidden
        
        # 출력 저장용
        outputs = torch.zeros(batch_size, max_length, vocab_size, device=self.device)
        predicted_indices = torch.zeros(batch_size, max_length, dtype=torch.long, device=self.device)
        
        for t in range(max_length):
            # 디코더 순전파
            output, decoder_hidden, attention = self.decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            
            # 출력 저장
            outputs[:, t, :] = output[:, 0, :]
            
            # 다음 토큰 예측
            top1 = output[:, 0, :].argmax(1)
            predicted_indices[:, t] = top1
            decoder_input = top1.unsqueeze(1)
            
            # EOS 토큰이 나오면 중단
            if (top1 == EOS_token).all():
                break
        
        return predicted_indices, outputs

In [25]:
def init_weights(model):
    """모델 가중치 초기화"""
    for name, param in model.named_parameters():
        if 'weight' in name:
            nn.init.xavier_uniform_(param)
        elif 'bias' in name:
            nn.init.constant_(param, 0)

In [26]:
# Seq2Seq 모델 생성
print("=== Seq2Seq 모델 생성 ===")
#seq2seq_model = Seq2SeqModel(encoder, decoder, device).to(device)
seq2seq_model = Seq2SeqModel(encoder, decoder, device).to(device)
init_weights(seq2seq_model)

print(f"모델 파라미터 수:")
total_params = sum(p.numel() for p in seq2seq_model.parameters())
trainable_params = sum(p.numel() for p in seq2seq_model.parameters() if p.requires_grad)
print(f"  총 파라미터: {total_params:,}")
print(f"  학습 가능한 파라미터: {trainable_params:,}")

=== Seq2Seq 모델 생성 ===
모델 파라미터 수:
  총 파라미터: 7,170,134
  학습 가능한 파라미터: 7,170,134


## 손실 함수 및 옵티마이저 설정

In [27]:
# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss(ignore_index=PAD_token)
optimizer = optim.Adam(seq2seq_model.parameters(), lr=0.001, weight_decay=1e-5)

# 학습률 스케줄러
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)


# Teacher Forcing 비율을 점진적으로 감소
def get_teacher_forcing_ratio(epoch, max_epochs):
    """에포치에 따라 Teacher Forcing 비율을 점진적으로 감소"""
    start_ratio = 0.9
    end_ratio = 0.1
    decay_rate = (start_ratio - end_ratio) / max_epochs
    return max(start_ratio - decay_rate * epoch, end_ratio)


print("=== 학습 설정 완료 ===")
print(f"손실 함수: {criterion}")
print(f"옵티마이저: {optimizer}")
print(f"학습률: {optimizer.param_groups[0]['lr']}")

=== 학습 설정 완료 ===
손실 함수: CrossEntropyLoss()
옵티마이저: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    initial_lr: 0.001
    lr: 0.001
    maximize: False
    weight_decay: 1e-05
)
학습률: 0.001


## 학습 함수 구현

In [28]:
def train_epoch(model, dataloader, criterion, optimizer, device, teacher_forcing_ratio=0.5):
    """한 에포크 학습"""
    model.train()
    total_loss = 0
    num_batches = len(dataloader)
    
    progress_bar = tqdm(dataloader, desc="Training")
    
    for batch_idx, batch in enumerate(progress_bar):
        # 데이터를 디바이스로 이동
        ko_input = batch['ko'].to(device)
        en_input = batch['en'].to(device)
        ko_lengths = batch['ko_lengths']
        en_lengths = batch['en_lengths']
        
        # 그래디언트 초기화
        optimizer.zero_grad()
        
        # 순전파
        outputs = model(ko_input, ko_lengths, en_input, teacher_forcing_ratio)
        
        # 손실 계산 - PAD 토큰 제외
        batch_size, seq_len, vocab_size = outputs.size()
        
        # PAD 토큰이 아닌 위치만 마스킹
        mask = (en_input != PAD_token).float()  # (B,S)
        
        # 마스킹된 손실 계산
        outputs_flat = outputs.view(-1, vocab_size)
        targets_flat = en_input.view(-1)
        mask_flat = mask.view(-1)
        
        # PAD가 아닌 위치만 손실 계산
        valid_positions = mask_flat > 0
        if valid_positions.sum() > 0:
            loss = criterion(
                outputs_flat[valid_positions], 
                targets_flat[valid_positions]
            )
        else:
            loss = torch.tensor(0.0, device=device, requires_grad=True)
        
        
        # 역전파
        loss.backward()
        
        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        # 파라미터 업데이트
        optimizer.step()
        
        # 손실 누적
        total_loss += loss.item()
        
        # 진행률 업데이트
        progress_bar.set_postfix({
            'Loss': f'{loss.item():.4f}',
            'Avg Loss': f'{total_loss/(batch_idx+1):.4f}'
        })
    
    return total_loss / num_batches

def validate_epoch(model, dataloader, criterion, device):
    """한 에포크 검증"""
    model.eval()
    total_loss = 0
    num_batches = len(dataloader)
    
    progress_bar = tqdm(dataloader, desc="Validation")
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(progress_bar):
            # 데이터를 디바이스로 이동
            ko_input = batch['ko'].to(device)
            en_input = batch['en'].to(device)
            ko_lengths = batch['ko_lengths']
            en_lengths = batch['en_lengths']
            
            # 순전파 (Teacher Forcing 없음)
            outputs = model(ko_input, ko_lengths, en_input, teacher_forcing_ratio=0.0)
            
            # 손실 계산 - PAD 토큰 제외
            batch_size, seq_len, vocab_size = outputs.size()
            
            # PAD 토큰이 아닌 위치만 마스킹
            mask = (en_input != PAD_token).float()  # (B,S)
            
            # 마스킹된 손실 계산
            outputs_flat = outputs.view(-1, vocab_size)
            targets_flat = en_input.view(-1)
            mask_flat = mask.view(-1)
            
            # PAD가 아닌 위치만 손실 계산
            valid_positions = mask_flat > 0
            if valid_positions.sum() > 0:
                loss = criterion(
                    outputs_flat[valid_positions], 
                    targets_flat[valid_positions]
                )
            else:
                loss = torch.tensor(0.0, device=device, requires_grad=True)
            
            # 손실 누적
            total_loss += loss.item()
            
            # 진행률 업데이트
            progress_bar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Avg Loss': f'{total_loss/(batch_idx+1):.4f}'
            })
    
    return total_loss / num_batches

## 번역 함수 구현

In [29]:
def translate_sentence(model, sentence, input_lang, output_lang, tokenizer_ko, tokenizer_en, max_length=50):
    """단일 문장 번역"""
    model.eval()
    
    # 한국어 문장을 인덱스로 변환
    ko_indexes = sentence_to_indexes(sentence, input_lang, tokenizer_ko, max_length)
    ko_tensor = torch.tensor(ko_indexes, dtype=torch.long).unsqueeze(0).to(device)
    ko_lengths = [len(tokenizer_ko(sentence))]
    
    with torch.no_grad():
        # 번역 수행
        predicted_indices, _ = model.predict(ko_tensor, ko_lengths, max_length)
        
        # 결과를 문장으로 변환
        translated_sentence = indexes_to_sentence(predicted_indices[0].cpu().numpy(), output_lang)
    
    return translated_sentence

def evaluate_translations(model, test_sentences, input_lang, output_lang, tokenizer_ko, tokenizer_en):
    """번역 결과 평가"""
    print("=== 번역 결과 평가 ===")
    
    for i, ko_sentence in enumerate(test_sentences[:5]):  # 처음 5개만 테스트
        print(f"\n예시 {i+1}:")
        print(f"  한국어: {ko_sentence}")
        
        # 번역 수행
        translated = translate_sentence(
            model, ko_sentence, input_lang, output_lang, 
            tokenizer_ko, tokenizer_en
        )
        print(f"  영어: {translated}")
        
        # 정답 확인
        if i < len(mt_sentences_train):
            print(f"  정답: {mt_sentences_train[i]}")

## 모델 학습

In [30]:
# 학습 파라미터 설정
EPOCHS = 20
SAVE_INTERVAL = 5

# 학습 및 검증 손실 저장
train_losses = []
valid_losses = []

print("=== 모델 학습 시작 ===")
print(f"총 에포크: {EPOCHS}")
print(f"체크포인트 저장 간격: {SAVE_INTERVAL} 에포크")

for epoch in range(EPOCHS):
    print(f"\n=== Epoch {epoch+1}/{EPOCHS} ===")
    
    # 학습
    teacher_forcing_ratio = get_teacher_forcing_ratio(epoch, EPOCHS)
    train_loss = train_epoch(seq2seq_model, train_loader, criterion, optimizer, device, teacher_forcing_ratio)
    #train_loss = train_epoch(seq2seq_model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    
    # 검증
    valid_loss = validate_epoch(seq2seq_model, valid_loader, criterion, device)
    valid_losses.append(valid_loss)
    
    # 학습률 조정
    scheduler.step()
    
    print(f"Epoch {epoch+1} 결과:")
    print(f"  훈련 손실: {train_loss:.4f}")
    print(f"  검증 손실: {valid_loss:.4f}")
    print(f"  학습률: {optimizer.param_groups[0]['lr']:.6f}")
    
    # 체크포인트 저장
    if (epoch + 1) % SAVE_INTERVAL == 0:
        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': seq2seq_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'train_loss': train_loss,
            'valid_loss': valid_loss,
            'train_losses': train_losses,
            'valid_losses': valid_losses
        }
        
        torch.save(checkpoint, f"checkpoints/seq2seq_attention_epoch_{epoch+1}.pth")
        print(f"  체크포인트 저장: epoch_{epoch+1}.pth")
    
    # 번역 결과 확인 (5에포크마다)
    if (epoch + 1) % 5 == 0:
        print("\n  번역 결과 확인:")
        test_sentences = ko_sentences_train[:3]  # 처음 3개 문장으로 테스트
        evaluate_translations(
            seq2seq_model, test_sentences, input_lang, output_lang, 
            tokenizer_ko, tokenizer_en
        )

print("\n=== 모델 학습 완료 ===")

=== 모델 학습 시작 ===
총 에포크: 20
체크포인트 저장 간격: 5 에포크

=== Epoch 1/20 ===


Validation: 100%|██████████| 32/32 [00:04<00:00,  7.04it/s, Loss=5.3864, Avg Loss=5.6769]


Epoch 1 결과:
  훈련 손실: 5.8304
  검증 손실: 5.6769
  학습률: 0.001000

=== Epoch 2/20 ===


Validation: 100%|██████████| 32/32 [00:03<00:00,  9.05it/s, Loss=5.3134, Avg Loss=5.7657]


Epoch 2 결과:
  훈련 손실: 5.1320
  검증 손실: 5.7657
  학습률: 0.001000

=== Epoch 3/20 ===


...

Epoch 18 결과:
  훈련 손실: 3.4907
  검증 손실: 5.3561
  학습률: 0.000500

=== Epoch 19/20 ===


Validation: 100%|██████████| 32/32 [00:03<00:00,  8.38it/s, Loss=5.0955, Avg Loss=5.4363]


Epoch 19 결과:
  훈련 손실: 3.5377
  검증 손실: 5.4363
  학습률: 0.000500

=== Epoch 20/20 ===


Validation: 100%|██████████| 32/32 [00:03<00:00,  8.77it/s, Loss=4.9524, Avg Loss=5.4185]


Epoch 20 결과:
  훈련 손실: 3.4679
  검증 손실: 5.4185
  학습률: 0.000250
  체크포인트 저장: epoch_20.pth

  번역 결과 확인:
=== 번역 결과 평가 ===

예시 1:
  한국어: 원하시는 색상을 회신해 주시면 바로 제작 들어가겠습니다.
  영어: If you have to the the , , to to to to to the the . .
  정답: If you reply to the color you want, we will start making it right away.

예시 2:
  한국어: 형님 제일 웃긴 그림이 뭔지 알아요.
  영어: What 's you , you you ? ?
  정답: I know what the funniest picture is.

예시 3:
  한국어: >속옷을?
  영어: Rock-paper-scissors .
  정답: Underwear?

=== 모델 학습 완료 ===


## 번역 품질 평가 및 분석

In [31]:
# 학습 완료 후 번역 품질 평가
def comprehensive_evaluation():
    print("=== 종합 번역 품질 평가 ===")
    
    # 1. 샘플 번역 결과
    test_sentences = ko_sentences_train[:10]
    evaluate_translations(seq2seq_model, test_sentences, input_lang, output_lang, tokenizer_ko, tokenizer_en)
    
    # 2. BLEU 점수 계산 (간단한 구현)
    bleu_score = calculate_simple_bleu()
    print(f"BLEU Score: {bleu_score:.2f}")
    
    # 3. Attention 가중치 시각화
    visualize_attention_weights()